In [ ]:
%pylab inline
from google.colab import drive
drive.mount('/content/drive')
# install OpenCV
!pip install opencv-python

# the usual ...
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage import img_as_float
import cv2
import math

Populating the interactive namespace from numpy and matplotlib
Mounted at /content/drive


In [ ]:
def GradientMag(img):
  img = img.astype('float32')
  blur = cv2.GaussianBlur(img, (3, 3), 0, 0)
  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
  sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
  sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
  ENERGY = cv2.add(np.absolute(sobelx), np.absolute(sobely))
  return ENERGY

def CEV(energy):
    height, width = energy.shape[:2]
    comul_energy = np.zeros((height, width))

    for i in range(1, height):
        for j in range(width):
            left = comul_energy[i - 1, j - 1] if j - 1 >= 0 else 1e6
            middle = comul_energy[i - 1, j]
            right = comul_energy[i - 1, j + 1] if j + 1 < width else 1e6
            
            comul_energy[i, j] = energy[i, j] + min(left, middle, right)

    return comul_energy

def vertical_seam(comul_energy):
    height, width = comul_energy.shape[:2]
    previous = 0
    seam = []
    for i in range(height - 1, -1, -1):
        row = comul_energy[i, :]

        if i == height - 1:
            previous = np.argmin(row)
            seam.append([previous, i])
        else:
            left = row[previous - 1] if previous - 1 >= 0 else 1e6
            middle = row[previous]
            right = row[previous + 1] if previous + 1 < width else 1e6

            previous = previous + np.argmin([left, middle, right]) - 1
            seam.append([previous, i])

    return seam